In [1]:
import torch
from dataloader import *
from CNN_Models import *
from CNN_Training import *
from model_io import *
from utils.constants import *

Configure Paths

In [2]:
current_path = ""
image_path = current_path + "data/images_activity"
label_path = current_path + "data/activity_label.csv"

Setting Up Correct Device

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
            "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
            "and/or you do not have an MPS-enabled device on this machine.")
else:
    device = torch.device("mps")
    
print(f"Currently Using Device: {device}")

Currently Using Device: mps


Obtain data loader, optimizer and model

In [ ]:
(train, valid) = get_activity_datasets(label_path, image_path, 16) # Use batch size of 16
model = get_resnet18_classifier(ACT_N_CLASSES, dropout=0.2) # Use smaller dropout

optimizer = get_optimizer(model, "sgd", lr = 1e-3, weight_decay = 1e-4, momentum = 0.9)
scheduler = get_scheduler(optimizer, "exponential", gamma = 0.95)

class_weights = torch.tensor([1,1,4,8,8,16,16,4], dtype = torch.float32).to(device) # Roughly invert class count
loss_criterion = get_loss(class_weights / torch.sum(class_weights)) 

Train the Model in Stages

In [11]:
# Save model checkpoint every 5 epochs
num_stages = 3
epoch_per_stage = 10
model_path = current_path + "model/"

# Initialize the logger object
logger = {"train_loss" : [], "valid_loss" : [], "train_acc" : [], "valid_acc" : []}


for stage in range(num_stages):
    CNN_train(model, train, valid, epoch_per_stage,
              loss_criterion, optimizer, scheduler, logger, True)
    model_save_name = model_path + f"model_stage{stage+1}.pth"
    save_checkpoint(model_save_name, model, optimizer, scheduler, logger)

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [MPSFloatType [16, 512, 7, 7]], which is output 0 of ReluBackward0, is at version 3; expected version 2 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).

Evaluating A Trained Model